<a href="https://colab.research.google.com/github/codml/AI_jupyter/blob/main/chapter_4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = (boston_housing.load_data())

import numpy as np

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std # 정규화에 쓰이는 평균, 표준편차도 훈련 데이터에서!

[3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]
[3.09233578e+00 1.09019608e+01 1.12649020e+01 9.80392157e-02
 5.44155882e-01 6.35415686e+00 6.68490196e+01 4.01198235e+00
 9.98039216e+00 4.17500000e+02 1.83745098e+01 3.64163333e+02
 1.23054902e+01]
[9.22929073e+00 2.37382770e+01 6.80287253e+00 2.40939633e-01
 1.17147847e-01 7.08908627e-01 2.79060634e+01 2.02770050e+00
 8.68758849e+00 1.66168506e+02 2.19765689e+00 9.39946015e+01
 7.24556085e+00]
[5.34668449e+00 2.14669193e+01 7.04933605e+00 2.97367664e-01
 1.09473981e-01 6.69031503e-01 2.88923144e+01 2.36827738e+00
 8.72901449e+00 1.76518774e+02 2.01682561e+00 7.87494364e+01
 6.66249874e+00]
